<a href="https://colab.research.google.com/github/sugar100/Section-Project/blob/main/AI_03_%EB%B0%B1%EC%84%A4_section04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **유튜브 조회수와 제목의 상관관계**
### 유튜브 제목에 포홤된 긍정/부정의 단어가 조회수에 영향을 미치는가?

## **데이터 불러오기 & 간단한 전처리**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd 
import numpy as np

import re
import string
import seaborn as sns
import matplotlib.pyplot as plt
import spacy
from spacy.tokenizer import Tokenizer
from sklearn.feature_extraction.text import TfidfVectorizer

from collections import Counter
from sklearn.model_selection import train_test_split, GridSearchCV

import tensorflow as tf
from tensorflow import keras

In [3]:
# Import Libraries

import glob
import json
import datetime as dt
import os
import string
import re    #for regex

import random

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

# plotting
import matplotlib.pyplot as plt
import matplotlib.pylab as pl
import matplotlib.colors as colors
import matplotlib.gridspec as gridspec
from matplotlib import cm
import seaborn as sns
import plotly.graph_objs as go
import plotly.offline as py

# natural language processing
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

import nltk
from nltk.util import bigrams
from nltk.util import trigrams
from nltk.corpus import stopwords
from nltk import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer 
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tokenize import RegexpTokenizer, TweetTokenizer


from wordcloud import WordCloud, STOPWORDS
from textblob import TextBlob

from tqdm import tqdm
import operator
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from statistics import *
import concurrent.futures
import time

from pylab import bone, pcolor, colorbar, plot, show, rcParams, savefig

# keras module for building LSTM 
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku 

In [4]:

#df(=youtube trending dataset) 불러오기
df = pd.read_csv('/content/drive/MyDrive/KRvideos.csv', engine = 'python')

# category name 생성
import json

with open('/content/drive/MyDrive/KR_category_id.json') as f:
  categories = json.load(f)["items"]
cat_dict = {}
for cat in categories:
  cat_dict[int(cat["id"])] = cat["snippet"]["title"]
df['category_name'] = df['category_id'].map(cat_dict)

df["category_name"].loc[df["category_name"].isnull() == True] = "비영리/사회운동"

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [5]:
# 필요없는 columns 삭제
df = df.drop(['video_id', 'thumbnail_link', 'comments_disabled', 'ratings_disabled', 'video_error_or_removed'], axis = 1)


In [6]:
df.duplicated().sum()  # 중복값 확인

# 중복값 제거
df = df.drop_duplicates()

In [7]:
df.isna().sum()

trending_date       0
title               0
channel_title       0
category_id         0
publish_time        0
tags                0
views               0
likes               0
dislikes            0
comment_count       0
description      2987
category_name       0
dtype: int64

In [8]:
df = df.dropna(axis=0)

# 자연어 전처리

### (1) Basic Preprocessing

In [9]:
df["title"] = df["title"].str.replace(pat=r'[^\w]', repl=r' ', regex=True)
df["tags"] = df["tags"].str.replace(pat=r'[^\w]', repl=r' ', regex=True)
df["description"] = df["description"].str.replace(pat=r'[^\w]', repl=r' ', regex=True)

### (2) Tokenize, 형태소 분석

In [10]:
pip install konlpy

In [11]:
from konlpy.tag import Okt 
okt = Okt()

In [12]:
 def getTokenedData(df):

    splited_input=[]
   
    for i in df['title']:
        temp_X =[]
        temp_X = okt.morphs(i, stem=True)
       

        splited_input.append(temp_X)

    return splited_input

In [13]:
df['title'] = getTokenedData(df)

In [14]:
 def getTokenedData2(df):

    splited_input=[]
   
    for i in df['tags']:
        temp_X =[]
        temp_X = okt.morphs(i, stem=True)
       

        splited_input.append(temp_X)

    return splited_input

In [15]:
df['tags'] = getTokenedData2(df)

In [16]:
 def getTokenedData3(df):

    splited_input=[]
   
    for i in df['description']:
        temp_X =[]
        temp_X = okt.morphs(i, stem=True)
       

        splited_input.append(temp_X)

    return splited_input

In [17]:
df['description'] = getTokenedData3(df)

In [18]:
df.head()

,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,description,category_name
0,17.14.11,"[좋다, by, 민서, _, 윤종신, _, 좋다, 답, 가]",라푸마코리아,22,2017-11-13T07:07:36.000Z,"[라, 푸마, 윤종신, 좋다, 좋다, 샬레, 민서]",156130,1422,40,272,"[윤종신, 좋다, 의, 답, 가, 좋다, 최초, 공개, n, 그, 여자, 의, 이야...",People & Blogs
1,17.14.11,"[JSA, 귀순, 북한, 군, 총격, 부상]",Edward,25,2017-11-13T10:59:16.000Z,"[JSA, 귀순, 북한, 군, 총격, 부상, JSA, 귀순, 북한, 군, 총격, 부상]",76533,211,28,113,"[채널, A, 단독, 北, 병사, 현재, 의식, 불명, 혈압, 떨어지다, 중, n,...",News & Politics
2,17.14.11,"[나, 모르다, 패밀리, 운동화, 영상, 2, 타다, 빼빼로데이, 버젼]",나몰라패밀리 핫쇼,22,2017-11-11T07:16:08.000Z,"[아디다스, 빼빼로, 핫쇼, 나, 모르다, 패밀리, 대학로, 공연]",421409,5112,166,459,"[퍼가실때, 꼭, 출처, 부탁드리다]",People & Blogs
3,17.14.11,"[이명박, 출국, 현장, 놓치다, 안되다, 장면]",미디어몽구,25,2017-11-12T11:19:52.000Z,"[이명박, 이명박, 출국금지, 이명박, 출국, 현장, 이명박, 바레인, 이명박, 대...",222850,2093,173,1219,"[다스, 는, 누구, 겁, 니까, 듣다, 난, 후, 표정]",News & Politics
4,17.14.11,"[김장, 겸, 은, 물르다, 가다, MBC, 노조, 환호, 와, 눈물]",NocutV,25,2017-11-13T11:08:59.000Z,"[nocutV, 노컷, V, CBS, mbc, 김장, 겸, 해임, 허일, 후]",84466,1094,109,450,"[김장, 겸, MBC, 사장, 이, 결국, 해임, 돼다, n, nMBC, 대주, 주...",News & Politics


## 상위, 하위 10개 유튜브 목록 

In [19]:
Q1 = df.views.quantile(0.25)
Q3 = df.views.quantile(0.75)
IQR = Q3 - Q1
print(IQR)

popular_videos=df.loc[df.views > (Q3 + 1.5 * IQR)]

df['popular']=0
df.loc[df.views > (Q3 + 1.5 * IQR),'popular']=1

df['popular'].value_counts()

237495.75


0    26012
1     3252
Name: popular, dtype: int64

In [20]:
#make a variable that tells ratio of like and dislike
df['like_percentage']=(df['likes']/(df['likes']+df['dislikes'])*100)
#date column as datatime datatype
df["publish_time"] = pd.to_datetime(df["publish_time"])

In [21]:
#좋아요 가장 많이 받은 10개 동영상
df.loc[df.popular==1].sort_values(by=['like_percentage'], ascending=False).head(10)

,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,description,category_name,popular,like_percentage
26902,18.01.05,"[G, C, F, in, Osaka]",BANGTANTV,10,2018-04-30 13:03:49+00:00,"[방탄소년단, BTS, BANGTAN, HIPHOP, 랩, 몬스터, RapMonst...",2335147,612761,568,56894,"[BTS, Official, Homepage, http, bts, ibighit, ...",Music,1,99.907391
27000,18.02.05,"[G, C, F, in, Osaka]",BANGTANTV,10,2018-04-30 13:03:49+00:00,"[방탄소년단, BTS, BANGTAN, HIPHOP, 랩, 몬스터, RapMonst...",2942269,688754,687,61516,"[BTS, Official, Homepage, http, bts, ibighit, ...",Music,1,99.900354
373,17.16.11,"[PREVIEW, BTS, 방탄소년단, 2018, Season, s, greetin...",BANGTANTV,10,2017-11-15 15:00:10+00:00,"[방탄소년단, BTS, BANGTAN, HIPHOP, 랩, 몬스터, RapMonst...",1002924,237304,299,11933,"[BTS, Official, Homepage, http, bts, ibighit, ...",Music,1,99.874160
706,17.17.11,"[PREVIEW, BTS, 방탄소년단, 2018, Season, s, greetin...",BANGTANTV,10,2017-11-15 15:00:10+00:00,"[방탄소년단, BTS, BANGTAN, HIPHOP, 랩, 몬스터, RapMonst...",1293307,276529,372,12886,"[BTS, Official, Homepage, http, bts, ibighit, ...",Music,1,99.865656
15037,18.12.02,"[Choreography, Video, SEVENTEEN, 세븐, 틴, 고맙다, T...",SEVENTEEN,10,2018-02-10 13:10:04+00:00,"[세븐, 틴, seventeen, 고맙다, thanks]",810425,118942,162,6417,"[Choreography, Video, SEVENTEEN, 세븐, 틴, 고맙다, T...",Music,1,99.863984
20913,18.20.03,"[Special, Trailer, BTS, Burn, The, Stage]",BANGTANTV,24,2018-03-20 03:00:00+00:00,"[방탄소년단, 슈가, 지민, 랩, 몬스터, 진, 정국, 제이, 홉, 뷔, 빅히트, ...",690491,210287,297,18786,"[Episode, 1, 2, release, on, March, 29, th, 12...",Entertainment,1,99.858964
2982,17.30.11,"[EPISODE, BTS, 방탄소년단, MIC, Drop, MV, Shooting]",BANGTANTV,10,2017-11-29 10:00:03+00:00,"[방탄소년단, BTS, BANGTAN, HIPHOP, 랩, 몬스터, RapMonst...",1447420,245076,354,12157,"[BTS, Official, Homepage, http, bts, ibighit, ...",Music,1,99.855763
34101,18.12.06,"[BTS, 방탄소년단, 방탄, 회식, 2018, BTSFESTA]",BANGTANTV,10,2018-06-11 15:00:03+00:00,"[방탄소년단, BTS, BANGTAN, HIPHOP, 랩, 몬스터, RapMonst...",2384801,416032,607,38992,"[BTS, Official, Homepage, http, bts, ibighit, ...",Music,1,99.854310
11190,18.20.01,"[BANGTAN, BOMB, Jin, s, Surprise, Birthday, Pa...",BANGTANTV,10,2018-01-19 11:00:05+00:00,"[방탄소년단, BTS, BANGTAN, HIPHOP, 랩, 몬스터, RapMonst...",1251010,218826,325,18005,"[BTS, Official, Homepage, http, bts, ibighit, ...",Music,1,99.851700
10083,18.14.01,"[천만원, 을, 일, 시불, 로, 긁다, 사연, 가조, 쿠, 들, 선물, 이야]",보겸 TV,20,2018-01-13 13:14:02+00:00,"[클래시, 로얄, clash, royale, 보겸, 보겸, TV, 슈, 마상, 도티...",773020,152329,228,249179,"[방학, 선물, 이야, 다, 들다, 잘, 받다, 댓글, 에, 메일, 주소, 남기다,...",Gaming,1,99.850548


In [22]:
#싫어요 가장 많이 받은 10개 동영상

df.loc[df.popular==1].sort_values(by=['like_percentage'], ascending=True).head(10)
                                                                              

,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,description,category_name,popular,like_percentage
5874,17.18.12,"[고영욱, 엄마, 자살, 근황, 사건, 추다, 전자발찌, 신상, 정보공개]",연예인,22,2017-12-15 10:00:04+00:00,"[KRVBA, TV, 연예인, 연예계, 스캔들, 고영욱, 고영욱, youtube]",761606,564,8551,1011,"[고영욱, 엄마, 자살, 근황, 사건, 추다, 전자발찌, 신상, 정보공개, n, 재...",People & Blogs,1,6.187603
16426,18.21.02,"[김보름, 진심, 으로, 사과, 드리다, 결국, 눈물, 로, 사죄, 기자회견, 풀,...",SBS 뉴스,25,2018-02-20 09:33:31+00:00,"[뉴스, NEWS, news, SBS, sbs, SBS, NEWS, SBS, 8, ...",1337500,2200,8092,14673,"[스피드스케이팅, 여자, 팀, 추월, 경기, 에서, 불거지다, 팀워크, 논란, 과,...",News & Politics,1,21.375826
182,17.15.11,"[현재, 논란중, 인, 홍대, 머리, 채남]",궁금타파,22,2017-11-12 18:52:16+00:00,"[홍대, 홍대, 버스킹]",1232807,872,2517,2185,"[홍대, 댄스, 버스킹, 하다, 갑자기, 여성, 분, 머리, 잡고, 나오다]",People & Blogs,1,25.730304
355,17.16.11,"[현재, 논란중, 인, 홍대, 머리, 채남]",궁금타파,22,2017-11-12 18:52:16+00:00,"[홍대, 홍대, 버스킹]",1528596,1208,3474,2883,"[홍대, 댄스, 버스킹, 하다, 갑자기, 여성, 분, 머리, 잡고, 나오다]",People & Blogs,1,25.800940
1225,17.20.11,"[현재, 논란중, 인, 홍대, 머리, 채남]",궁금타파,22,2017-11-12 18:52:16+00:00,"[홍대, 홍대, 버스킹]",1799687,1517,4360,3316,"[홍대, 댄스, 버스킹, 하다, 갑자기, 여성, 분, 머리, 잡고, 나오다]",People & Blogs,1,25.812489
986,17.19.11,"[현재, 논란중, 인, 홍대, 머리, 채남]",궁금타파,22,2017-11-12 18:52:16+00:00,"[홍대, 홍대, 버스킹]",1788852,1506,4323,3307,"[홍대, 댄스, 버스킹, 하다, 갑자기, 여성, 분, 머리, 잡고, 나오다]",People & Blogs,1,25.836336
748,17.18.11,"[현재, 논란중, 인, 홍대, 머리, 채남]",궁금타파,22,2017-11-12 18:52:16+00:00,"[홍대, 홍대, 버스킹]",1763874,1487,4264,3295,"[홍대, 댄스, 버스킹, 하다, 갑자기, 여성, 분, 머리, 잡고, 나오다]",People & Blogs,1,25.856373
21579,18.24.03,"[이태임, 은퇴, 이태임, 이명박, 관계, 비밀, 이, 밝혀지다]",Korea News 24H,24,2018-03-20 06:00:02+00:00,"[이태임, 은퇴, 이태임, 이명박, 관계, 비밀, 이, 밝혀지다, youtube, ...",989744,991,2836,463,"[이태임, 은퇴, 이태임, 이명박, 관계, 비밀, 이, 밝혀지다, 이태임, 이명박,...",Entertainment,1,25.894957
21303,18.23.03,"[이태임, 은퇴, 이태임, 이명박, 관계, 비밀, 이, 밝혀지다]",Korea News 24H,24,2018-03-20 06:00:02+00:00,"[이태임, 은퇴, 이태임, 이명박, 관계, 비밀, 이, 밝혀지다, youtube, ...",923536,953,2727,462,"[이태임, 은퇴, 이태임, 이명박, 관계, 비밀, 이, 밝혀지다, 이태임, 이명박,...",Entertainment,1,25.896739
546,17.17.11,"[현재, 논란중, 인, 홍대, 머리, 채남]",궁금타파,22,2017-11-12 18:52:16+00:00,"[홍대, 홍대, 버스킹]",1671363,1393,3958,2994,"[홍대, 댄스, 버스킹, 하다, 갑자기, 여성, 분, 머리, 잡고, 나오다]",People & Blogs,1,26.032517


## TfidfVectorizer

In [23]:
tokenizer = Tokenizer()

def get_sequence_of_tokens(corpus):
    ## tokenization
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1
    
    ## convert data to sequence of tokens 
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences, total_words

inp_sequences, total_words = get_sequence_of_tokens(df['title'])
inp_sequences[:10]

[[199, 146],
 [199, 146, 3644],
 [199, 146, 3644, 23],
 [199, 146, 3644, 23, 2342],
 [199, 146, 3644, 23, 2342, 23],
 [199, 146, 3644, 23, 2342, 23, 199],
 [199, 146, 3644, 23, 2342, 23, 199, 1295],
 [199, 146, 3644, 23, 2342, 23, 199, 1295, 6],
 [1754, 580],
 [1754, 580, 80]]

In [26]:
def generate_padded_sequences(input_sequences):
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
    
  
    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = tf.keras.utils.to_categorical(label, num_classes=total_words)
    return predictors, label, max_sequence_len

predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences[:1000])

In [27]:

def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    
    # Add Input Embedding Layer
    model.add(Embedding(total_words, 10, input_length=input_len))
    
    # Add Hidden Layer 1 - LSTM Layer
    model.add(LSTM(100))
    model.add(Dropout(0.1))
    
    # Add Output Layer
    model.add(Dense(total_words, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam')
    
    return model

model = create_model(max_sequence_len, total_words)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 27, 10)            241500    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               44400     
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 24150)             2439150   
Total params: 2,725,050
Trainable params: 2,725,050
Non-trainable params: 0
_________________________________________________________________


In [28]:
model.fit(predictors, label, epochs=50, verbose=5)

Epoch 1/50
Epoch 2/50
Epoch 3/50
Epoch 4/50
Epoch 5/50
Epoch 6/50
Epoch 7/50
Epoch 8/50
Epoch 9/50
Epoch 10/50
Epoch 11/50
Epoch 12/50
Epoch 13/50
Epoch 14/50
Epoch 15/50
Epoch 16/50
Epoch 17/50
Epoch 18/50
Epoch 19/50
Epoch 20/50
Epoch 21/50
Epoch 22/50
Epoch 23/50
Epoch 24/50
Epoch 25/50
Epoch 26/50
Epoch 27/50
Epoch 28/50
Epoch 29/50
Epoch 30/50
Epoch 31/50
Epoch 32/50
Epoch 33/50
Epoch 34/50
Epoch 35/50
Epoch 36/50
Epoch 37/50
Epoch 38/50
Epoch 39/50
Epoch 40/50
Epoch 41/50
Epoch 42/50
Epoch 43/50
Epoch 44/50
Epoch 45/50
Epoch 46/50
Epoch 47/50
Epoch 48/50
Epoch 49/50
Epoch 50/50


In [29]:
def generate_text(seed_text, next_words):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict_classes(token_list, verbose=0)
        
        output_word = ""
        for word,index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " "+output_word
    return seed_text.title()

In [30]:
def generate_title_texts(words_list):
    for word in words_list:
        print(generate_text(word, random.randint(4, 10)))